In [1]:
from kafka import KafkaConsumer # type: ignore
import datetime
from datetime import datetime
import ast
%run 03_HDFS_Data_lake.ipynb
%run 05_utils.ipynb

In [2]:
consumer_2 = KafkaConsumer("sensors_data")
print("Connection to sensors_data topic has been created successfuly")
client = hdfs_InsecureClient() # type: ignore

Connection to sensors_data topic has been created successfuly
HDFS Insecure Client has been created and returned successfuly


In [3]:
now = datetime.now()
file_path = "/Sensors_DL/{}/{}/day_{}.csv".format(now.year,now.month,now.day)

In [4]:
try:
    with client.write(file_path,encoding='utf-8',overwrite=False) as writer:
        for msg in consumer_2:
            current_time = datetime.now()
            if current_time.day != now.day or current_time.month != now.month or current_time.year != now.year:
                # If the day has changed, close the current file and create a new one
                writer.close() 
                now = current_time
                file_path = "/Sensors_DL/{}/{}/day_{}.csv".format(now.year,now.month,now.day)
                   
                # Extract and decode information from kafka consumer 
                message_str = msg.value.decode("utf-8")
                message = ast.literal_eval(message_str)
                data = "{},{},{},{},{} \n".format(*list(message.values()))
                writer.write(data)  
            # Extract and decode information from kafka consumer 
            message_str = msg.value.decode("utf-8")
            message = ast.literal_eval(message_str)
            data = "{},{},{},{},{} \n".format(*list(message.values()))
            writer.write(data)    
except KeyboardInterrupt:
    pass
except:
    pass

In [ ]:
client.delete("/Sensors_DL/2025/6/day_19.csv",recursive=True)

#client.delete("/Daily_reports",recursive=True)

True